In [ ]:
# Possible Idea: create 3D mapping of wind turbine s.t. we can make a 3D map of where the damage is

In [1]:
import os
import torch
import numpy as np
from WTDataset import WTDataset
from collections import Counter
from PIL import Image, ImageDraw
import plotly.graph_objects as go

In [2]:
data = WTDataset('data/NordTank586x371')
print(data)

In [3]:
data.__len__()
data.__getitem__(5)

(<PIL.PngImagePlugin.PngImageFile image mode=RGB size=586x371>,
 {'labels': tensor([1., 1., 1., 1., 1.], dtype=torch.float64),
  'boxes': tensor([[0.0546, 0.8693, 0.0785, 0.9097],
          [0.1604, 0.7574, 0.1843, 0.7790],
          [0.1962, 0.9245, 0.2270, 0.9677],
          [0.2952, 0.9717, 0.3362, 1.0283],
          [0.4667, 0.9879, 0.5060, 1.0121]], dtype=torch.float64),
  'img_id': tensor([5]),
  'area': tensor([0.0010, 0.0005, 0.0013, 0.0023, 0.0010], dtype=torch.float64),
  'iscrowd': tensor([0, 0, 0, 0, 0])})

## EDA Images

In [4]:
# Check if all images have the same width and height
w, h = [], []
for img in os.listdir(data.imgs):
    im = Image.open(os.path.join(data.imgs, img))
    w.append(im.size[0])
    h.append(im.size[1])

In [5]:
# All images have shape (586, 371)
print(Counter(h))

Counter({371: 13470})


In [6]:
# Number of labeled images vs unlabeled images
print("Total number of images:", len(w))
print("Total number of annotated images:", len(os.listdir(data.labels)))

Total number of images: 13470
Total number of annotated images: 2996


## EDA Labels and Bounding boxes

In [7]:
# Compute the amount of dirt and damage bounding boxes per image
dirt, damage = [], []
for item in data.data_list:
    target = item['target']
    
    # Only take into account annotated images
    if target['boxes'].shape[1] != 0:
        num_damage = len(torch.eq(target['labels'], 1).nonzero())
        dirt.append(target['labels'].shape[0] - num_damage)
        damage.append(num_damage)

In [8]:
# Plot the amount of dirt and damage bounding boxes per image (for annotated images)
fig = go.Figure()
fig.add_trace(go.Histogram(x=damage, name="Damage", marker_color='#DEA0FD'))
fig.add_trace(go.Histogram(x=dirt, name="Dirt", marker_color='#AA0DFE'))

fig.update_layout(
    title_text='Number of bounding boxes per image',
    xaxis_title_text='Number of bounding boxes',
    yaxis_title_text='Count',
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()

In [9]:
# Compute the aspect ratios of the bounding boxes
ars, labels = [], []
for data_pair in data.data_list:
    boxes = data_pair["target"]["boxes"]
    if boxes.shape[1] != 0:
        ar = 1.0 * (np.abs(boxes[:, 0] - boxes[:, 2]) / np.abs(boxes[:, 1] - boxes[:, 3]))
        ars += list(ar)
        labels += list(data_pair["target"]["labels"])

In [12]:
print(labels.count(1))

8770


In [ ]:
# Plot the aspect ratios of the bounding boxes
fig = go.Figure(data=[go.Histogram(x=ars, marker_color='#AA0DFE')])

fig.update_layout(
    title_text='The aspect ratios of the bounding boxes',
    xaxis_title_text='Number of bounding boxes',
    yaxis_title_text='Count',
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()

In [ ]:
# Plot the amount of dirt and damage labels
fig = go.Figure(data=[go.Bar(x=['Dirt', 'Damage'], y=[labels.count(2), labels.count(1)], marker_color='#AA0DFE')])
fig.update_layout(
    title_text='Count dirt and damage labels',
    yaxis_title_text='Count'
)
fig.show()

In [ ]:
# Find the corner points for drawing the boxes
def draw(box, img_shape):
    x0  = (box[0] - np.abs(box[0] - box[2]) / 2.) * img_shape[0]
    x1 = (box[0] + np.abs(box[0] - box[2]) / 2.) * img_shape[0]
    y0   = (box[1] - np.abs(box[1] - box[3]) / 2.) * img_shape[1]
    y1   = (box[1] + np.abs(box[1] - box[3]) / 2.) * img_shape[1]
    
    x0 = max(x0, 0)
    y0 = max(y0, 0)
    x1 = min(x1, img_shape[0] - 1)
    y1 = min(y1, img_shape[1] - 1)
    
    return x0, x1, y0, y1

In [ ]:
# Create a heatmap of all bounding box locations in the images
heatmap_dirt = np.zeros((371, 586), dtype=int)
heatmap_damage = np.zeros((371, 586), dtype=int)
for pair in data.data_list:
    boxes = pair['target']['boxes']
    if boxes.size()[1] != 0:
        for i, box in enumerate(boxes):
            l, r, t, b = draw(box, (586, 371))
            if pair["target"]["labels"][i] == 1:
                heatmap_damage[int(t):int(b), int(l):int(r)] += 1
            else:
                heatmap_dirt[int(t):int(b), int(l):int(r)] += 1

In [ ]:
# Plot heatmap of bounding boxes
fig = go.Figure(data=go.Heatmap(
                    z=heatmap_damage))
fig.update_layout(
    title_text='Heatmap of the locations of the bounding boxes'
)
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

In [ ]:
# Plot heatmap of bounding boxes
fig = go.Figure(data=go.Heatmap(
                    z=heatmap_dirt))
fig.update_layout(
    title_text='Heatmap of the locations of the bounding boxes'
)
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

In [ ]:
# Check all outliers of the aspect ratio histogram if the bboxes can be used
for pair in data.data_list:
    boxes = pair['target']['boxes']
    if boxes.size()[1] != 0:
        ar = 1.0 * (np.abs(boxes[:, 0] - boxes[:, 2]) / np.abs(boxes[:, 1] - boxes[:, 3]))
        if torch.any(ar > 4):
            img = Image.open(os.path.join(data.imgs, pair['img_name']))
            draw_img = ImageDraw.Draw(img)
            for box in boxes:
                l, r, t, b = draw(box, img.size)
                draw_img.rectangle([l, t, r, b], outline ="red", width=1)

            img.show()
#             img.save(pair['img_name'][:-4] + "_bboxes.PNG")